# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [6]:
import sys
sys.path.append('..')
import os
import optuna
from tsai.optuna import *
from tsai.basics import load_object
import papermill as pm
from tsai.optuna import run_optuna_study
from fastcore.basics import *
from optuna.distributions import *
from optuna.samplers import TPESampler
import wandb

In [7]:
config = AttrDict(
    study_name = 'general_study_extended', # name of the Optuna study
    study_type = 'bayesian', # 'bayesian' or 'gridsearch' or 'random'
    n_trials = 50, # number of trials
    train_nb = f'{os.getcwd()}/geodstap_train.ipynb', # path to the notebook to be executed
    search_space = {
        "arch.attn_dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1),
        "arch.d_model": IntUniformDistribution(32, 512, 32),
        "arch.d_ff": IntUniformDistribution(32, 512, 32),
        "arch.decomposition": CategoricalDistribution([True, False]),
        "arch.dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1), 
        "arch.individual": CategoricalDistribution([True, False]),
        "arch.n_layers": IntUniformDistribution(1, 6, 1),
        "arch.n_heads": CategoricalDistribution([2, 4, 8, 16, 32]),
        "init_weights": CategoricalDistribution([True, False]), # true = kaiming
        "lookback": CategoricalDistribution([144, 192, 240, 288, 336]), # horizon * {3,4,5,6,7,8}
        # "deltaHL": FloatDistribution(1., 15., step=1.),
    },
    # Add extra parameters that are fixed, but not part of the search space
    extra_params = {
        "n_epoch": 50,
        "bs": 192,
        "is_optuna_study": True
    },
    use_wandb = True,
    wandb_mode = 'offline'
)

config

```json
{ 'extra_params': {'bs': 192, 'is_optuna_study': True, 'n_epoch': 50},
  'n_trials': 50,
  'search_space': { 'arch.attn_dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.d_ff': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.d_model': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.decomposition': CategoricalDistribution(choices=(True, False)),
                    'arch.dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.individual': CategoricalDistribution(choices=(True, False)),
                    'arch.n_heads': CategoricalDistribution(choices=(2, 4, 8, 16, 32)),
                    'arch.n_layers': IntUniformDistribution(high=6, low=1, step=1),
                    'init_weights': CategoricalDistribution(choices=(True, False)),
                    'lookback': CategoricalDistribution(choices=(144, 192, 240, 288, 336))},
  'study_name': 'general_study_extended',
  'study_type': 'bayesian',
  'train_nb': '/workspaces/sw-driver-forecaster/dev_nbs/geodstap_train.ipynb',
  'use_wandb': True,
  'wandb_mode': 'offline'}
```

In [8]:
def create_objective(train_nb, search_space, extra_params=None, use_wandb=False):
    """
        Create objective function to be minimized by Optuna.
        Inputs:
            trial: Optuna trial object
            train_nb: path to the training notebook
            search_vars: keys of the search space to be used
            wandb_group: name of the wandb group to be used
        Output:
            valid_loss: validation loss
    """
    def objective(trial:optuna.Trial):
        # Define the parameters to be passed to the training notebook through papermill
        pm_parameters = {}
        for k,v in search_space.items():
            pm_parameters['config.' + k] = trial._suggest(k, v)

        # Add the extra parameters to the dictionary. The key of every parameter 
        # must be 'config.<param_name>'
        if extra_params is not None:
            for k,v in extra_params.items():
                pm_parameters['config.' + k] = v
                
        # If using wandb, enable that in the training runs, all of them gathered
        # into a group (NOTE: The train nb must have and use these config arguments)
        if use_wandb:
            pm_parameters['config.use_wandb'] = True
            pm_parameters['config.wandb_group'] = config.study_name + '_runs'

        # Call the training notebook using papermill (don't print the output)
        stdout_file = open('tmp/pm_stdout.txt', 'w')
        stderr_file = open('tmp/pm_stderr.txt', 'w')

        pm.execute_notebook(
            train_nb,
            './tmp/pm_output.ipynb',
            parameters = pm_parameters,
            stdout_file = stdout_file,
            stderr_file = stderr_file,
            
        )

        # Close the output files
        stdout_file.close()
        stderr_file.close()

        # Get the output value of interest from the source notebook
        loss = None
        %store -r valid_loss
        return valid_loss

    return objective

In [9]:
import nbformat

def filter_nb (path:str, skip_tags:str):
    # Load the notebook
    nb = nbformat.read(path, as_version=4)

    # Filter out cells with specific tags
    filtered_cells = [cell for cell in nb.cells if not set(skip_tags) & set(cell.metadata.get('tags', []))]
    nb.cells = filtered_cells
    
    return nb

In [10]:
# Skip the cells that are unnecesary as diagram plots or data update
train_nb = filter_nb(config.train_nb, 'skip')

obj = create_objective(train_nb, config.search_space, 
                       extra_params=config.extra_params, use_wandb=True)
study = run_optuna_study(obj, study_type='bayesian', direction='minimize', path='./tmp',
                 study_name=config.study_name, n_trials=config.n_trials)

[I 2024-08-13 11:37:49,125] A new study created in memory with name: general_study_extended
  0%|          | 0/50 [00:00<?, ?it/s]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


Best trial: 0. Best value: 53.8745:   2%|▏         | 1/50 [01:08<55:40, 68.17s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


[I 2024-08-13 11:38:57,297] Trial 0 finished with value: 53.8745002746582 and parameters: {'arch.attn_dropout': 0.30000000000000004, 'arch.d_model': 288, 'arch.d_ff': 512, 'arch.decomposition': False, 'arch.dropout': 0.30000000000000004, 'arch.individual': True, 'arch.n_layers': 3, 'arch.n_heads': 32, 'init_weights': False, 'lookback': 288}. Best is trial 0 with value: 53.8745002746582.


Best trial: 0. Best value: 53.8745:   4%|▍         | 2/50 [02:24<58:09, 72.69s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


[I 2024-08-13 11:40:13,156] Trial 1 finished with value: 56.3623046875 and parameters: {'arch.attn_dropout': 0.30000000000000004, 'arch.d_model': 512, 'arch.d_ff': 256, 'arch.decomposition': True, 'arch.dropout': 0.2, 'arch.individual': False, 'arch.n_layers': 2, 'arch.n_heads': 2, 'init_weights': True, 'lookback': 240}. Best is trial 0 with value: 53.8745002746582.


...

Best trial: 18. Best value: 50.8668:  98%|█████████▊| 49/50 [49:40<00:40, 40.98s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


[I 2024-08-13 12:27:29,858] Trial 48 finished with value: 52.53421401977539 and parameters: {'arch.attn_dropout': 0.0, 'arch.d_model': 192, 'arch.d_ff': 96, 'arch.decomposition': True, 'arch.dropout': 0.5, 'arch.individual': False, 'arch.n_layers': 1, 'arch.n_heads': 2, 'init_weights': True, 'lookback': 192}. Best is trial 18 with value: 50.866783142089844.


Best trial: 18. Best value: 50.8668: 100%|██████████| 50/50 [50:23<00:00, 60.47s/it]

[I 2024-08-13 12:28:12,501] Trial 49 finished with value: 51.7860107421875 and parameters: {'arch.attn_dropout': 0.1, 'arch.d_model': 224, 'arch.d_ff': 160, 'arch.decomposition': True, 'arch.dropout': 0.5, 'arch.individual': True, 'arch.n_layers': 1, 'arch.n_heads': 8, 'init_weights': True, 'lookback': 288}. Best is trial 18 with value: 50.866783142089844.

Optuna study saved to tmp/general_study_extended.pkl
To reload the study run: study = joblib.load('tmp/general_study_extended.pkl')



Study statistics    : 
  Study name        : general_study_extended
  # finished trials : 50
  # pruned trials   : 0
  # complete trials : 50

Best trial          :
  value             : 50.866783142089844
  best_params = {'arch.attn_dropout': 0.4, 'arch.d_model': 512, 'arch.d_ff': 32, 'arch.decomposition': False, 'arch.dropout': 0.5, 'arch.individual': True, 'arch.n_layers': 2, 'arch.n_heads': 2, 'init_weights': True, 'lookback': 240}



In [ ]:
# os.environ['WANDB_NOTEBOOK_NAME'] = '/workspaces/sw-driver-forecaster/dev_nbs/optuna_study_geodstap.ipynb'

run = wandb.init(config=config, mode=config['wandb_mode'], job_type='optuna-study') if config['use_wandb'] else None

In [ ]:
if run is not None:
    run.log(dict(study.best_params, **{'best_value': study.best_value, 
                                       'best_trial_number': study.best_trial.number}))
    run.log_artifact(f'./tmp/{config.study_name}.pkl', type='optuna_study')
    run.log({
        'contour': optuna.visualization.plot_contour(study),
        'edf': optuna.visualization.plot_edf(study),
        'intermediate_values': optuna.visualization.plot_intermediate_values(study),
        'optimization_history': optuna.visualization.plot_optimization_history(study),
        'parallel_coordinate' : optuna.visualization.plot_parallel_coordinate(study),
        'param_importances': optuna.visualization.plot_param_importances(study),
        'slice': optuna.visualization.plot_slice(study)
    })

In [ ]:
if run is not None:
    run.finish()

arch.attn_dropout,▁
arch.d_ff,▁
arch.d_model,▁
arch.decomposition,▁
arch.dropout,▁
arch.individual,▁
arch.n_heads,▁
arch.n_layers,▁
best_trial_number,▁
best_value,▁
init_weights,▁
